Welcome to week 7 of the [Noisebridge Python class](https://github.com/audiodude/PythonClass)!

In this lesson, we will discuss web scraping. While you may not have an immediate use for the techniques outlined here, it is worthwhile to learn in order to get a better understanding of the structure of the web and the possibilities for extracting data from websites.

We will be using [requests](https://requests.readthedocs.io/en/latest/) and [Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/). There are lots of other solutions for web scraping out there, including those written in other languages and hosted in cloud services.

In this lesson, you will learn:

* How to get the raw source code of websites using their URL.
* How to parse that source code into a data structure.
* How to identify and refer to specific locations inside of that data structure that contain useful information.
* *How to save the data in a Pandas DataFrame for easy querying, manipulation and export to CSV*

The Mozilla Developer Network has [an article](https://developer.mozilla.org/en-US/docs/Learn/Getting_started_with_the_web/How_the_Web_works) about the basic way the web works. Basically, when you type a URL into your browser and hit 'enter', the following happens:

1. The browser looks up the DNS address (a number) that corresponds to the domain in the URL.
1. The browser sends an HTTP GET request to the server (there are other HTTP "verbs" other than GET as well).
1. The server locates the [HTML](https://en.wikipedia.org/wiki/HTML) file you requested (in the case of a `example.com/foo/bar.html` type URL, where bar.html is the file) or communicates with an application to generate an HTML response, and sends it back to the browser over the [TCP/IP](https://www.cloudflare.com/learning/ddos/glossary/tcp-ip/) connection.
1. The browser receives the response, parses it, and renders a webpage based on the received HTML along with [CSS](https://developer.mozilla.org/en-US/docs/Web/CSS) and [Javascript](https://developer.mozilla.org/en-US/docs/Web/JavaScript).

HTML, CSS and Javascript are the foundational pieces of the source code of the web. HTML provides the structure for a webpage, in the form of an HTML document. HTML documents are composed of tags such as `<html>` (the wrapper for the whole document) and `<a>` (a link in the document to somewhere else). CSS represents the presentation logic for the page, from things as basic as colors and margins, to complex concerns of layout and formatting. Finally, Javascript provides a programming language which allows for dynamic effects and interactions inside of a webpage. For example, when you type a query into a search engine and it provides "auto complete" or suggested results. Those results are retrieved and displayed with Javascript.

Let's look at a simple website from NPR, https://text.npr.org/

![Screenshot of NPR basic "text" news site](https://pixelfed.social/storage/m/_v2/588554065884192073/bf0f52ff2-92677b/e5wi2rcyLEpI/RftouyiJETFbf1rKHXWTDMArbpHfMVOtZTjfMLfw.png)

We can use the program `curl` on macOS or Linux to retrieve the raw HTML contents of a web page. For example, when we run:

`$ curl https://text.npr.org/`

We get:

![Screenshot of the result of running curl on text.npr.org](https://pixelfed.social/storage/m/_v2/588554065884192073/bf0f52ff2-92677b/QcxYbe5gcGJE/pFJs6XokUP7rgLRbRz76ak3MchCMmVZqZoIILKTA.png)

and if we scroll further down we see:

![Screenshot of the result of running curl on text.npr.org](https://pixelfed.social/storage/m/_v2/588554065884192073/bf0f52ff2-92677b/2YqKQi3pOe4z/R2N4ZlWO8ht0hmTS1mwiBXrx1a2GZPt0Xl9zFJKp.png)

We see that the first news item in the web page, "How climate change could cause a home insurance meltdown", corresponds to HTML code in the curl response:

```
        <li><a class="topic-title" href="/1186540332">How climate change could cause a home insurance meltdown</a></li>
```

This is the "raw" code that the browser reads to create the web page seen in the first screenshot.

The important thing to understand is that `curl` is retrieving the source code of the webpage in the exact same way your web browser is, except that the browser takes the extra step of turning the textual HTML data into a graphical web page.

The `<li>` tag means that the article is part of a list, and the `<a>` tag means that the article name should be rendered as as link, which leads to the article.

We can also get the source code of the NPR text news website in Python using the `requests` library, which is for HTTP requests. The result is the same HTML code that we saw in the `curl` example.

In [ ]:
import requests

base_url = 'https://text.npr.org/'

resp = requests.get(base_url)
# Throw an exception if we get a non-success status code.
resp.raise_for_status()

code = resp.text

print(code[:195])

When you make a request to an HTTP server, the request includes, in it's metadata, an **HTTP status code**. You may have already heard of famous ones like "404 NOT FOUND" or "500 SERVER ERROR". You can find a list of status codes [on Wikipedia](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).

In general, 1xx status codes are for HTTP specific information. 2xx status codes generally indicate success, so we can proceed with processing the response. 3xx generally represent a **redirect**, where the HTTP server is instructing the client to make a new request to a different location (if you've ever typed a URL into your browser and wound up on a wildly different URL, that's probably a 302 redirect). The 4xx status codes are request errors, which generally mean the client did something wrong with the request such as 404 (requested something that doesn't exist) or the generic 400 (perhaps made a search request without a query or something). Finally, 5xx status codes represent something going wrong on the server, that is beyond the client's control.

The `requests` method `.raise_for_status()` will raise an exception if any non-2xx status code is encountered (except for redirects, 301/302 which are followed). This prevents us from proceeding and using a possibly nonsensical (to us) response.

---

Let's say we wanted to get a list of all news story headlines from this page. At this point, we could attempt to extract that data from the textual representation of the HTML (the raw code) using regex or simple string find operations.

In [ ]:
li_idx = code.find('<li>')
li_end_idx = code.find('</li>', li_idx)
print(code[li_idx:li_end_idx+5])

next_li_idx = code.find('<li>', li_end_idx)
next_li_end_idx = code.find('</li>', next_li_idx)
print(code[next_li_idx:next_li_end_idx+5])

However this approach is tricky and error prone. In this example, it's particularly simple, because each of the `<li>` elements contains exactly one `<a>` element. However in other cases, there could be nested structures that only occur in some of the desired target elements. For this reason, we will **parse** the HTML into a **document tree** and use the tree data structure to access the data we're interested in.

In [ ]:
from bs4 import BeautifulSoup

# The variable soup is our parsed document tree data structure.
# 'lxml' specifies the parser library to use. It must be installed.
soup = BeautifulSoup(code, 'lxml')

soup.find_all('a')

This is much easier, and seems very promising. We seem to have gotten all of the article links! But there's a problem, we're also getting links to the "Terms of Use" and "Go to Full Site".

Now we enter the part of web scraping that is more of an art than a science, and that is finding unique identifiers for data we care about, and navigating the document tree.

*Can you see a pattern with the links we care about (the news articles) that we could potentially use to extract just those?*

In [ ]:
# We use class_ with an underscore because `class` is a keyword in Python.
articles = soup.find_all('a', class_='topic-title')
articles

At this point we need to extract the actual article titles and dispense with the markup. We can use the `.text` attribute of the returned `Tag` objects. This attribute actually abstracts some of the details of dealing with what are called "Text Nodes" in HTML documents, where a single logical chunk of text might be split across multiple nodes or contain embedded HTML tags. For example, this HTML:

```
<a>Hello, I <em>love</em> learning Python!</a>
```

Contains mutliple text nodes under the `<a>` tag. However, using `.text` we can get all of the text under the tag, as we would expect.

In [ ]:
s2 = BeautifulSoup('<a>Hello, I <em>love</em> learning Python!</a>', 'lxml')
s2.a.text

*What is a list comprehension that we could use to find all of the article title strings from the `articles` tags?*

In [17]:
[a.text for a in articles]
print('Done')

NameError: name 'articles' is not defined

And that's it, that's web scraping. Lesson over, go home! (kidding not kidding)

Imagine that twitter bots were still a thing, or better yet you wanted to post to Mastodon (as we did in lesson 1) with a post every time a new article was published to the NPR text website. You could set up a scraper that runs every 5 minutes and if it finds an article title it hasn't seen before, it creates a post with a link to the article. But how do we get the link to the article? It's in the `href` **attribute** of the `<a>` tag:

In [ ]:
articles[0].get('href')

This is a **relative url**. It's not a full website address. It leads to a document that is at a path relative to the `base_url` of the website. Let's look again at `base_url`:

In [ ]:
base_url

You might consider that you could simply concatenate the `base_url` to the relative URL. However this is error prone and doesn't account for particular edge cases.

In [ ]:
base_url + articles[0].get('href')[1:]

By using a function called `urljoin` in the `urllib.parse` module, we can reconstruct the full URL.

In [ ]:
import urllib.parse

first_url = urllib.parse.urljoin(base_url, articles[0].get('href'))
first_url

We could then do:

```
post_to_mastodon('NPR: %s (%s)' % (articles[0].text, first_url))
```

---

Now let's try a more complex exercise. Let's say we want to build a data set of movie data from Rotten Tomatoes. For this example, we will consider 90s Science Fiction movies. To even begin, we will need a list of such movies, which we will get to later. First let's verify that we're able to retrieve the data we want from the Rotten Tomatoes website. Some websites are constructed dynamically using Javascript, which means that the information that eventually shows in your browser is not present in the initial HTML response. If that were the case with Rotten Tomatoes, we'd be unable to scrape the site using the methods we've discussed.

Let's look at the page for the 1990 sci-fi movie *Total Recall* (https://www.rottentomatoes.com/m/total_recall).

![Rotten tomatoes page for Total Recall](https://pixelfed.social/storage/m/_v2/588554065884192073/bf0f52ff2-92677b/PT9NMp8C3F5b/sI3iHqHumV9eGeOIFsPMr65hkMF3UTNnuhTxtknB.png)

We can see the "Tomatometer" rating there, as well as information such as the rating, the genre and the runtime. All of this would be useful information. Let's "view source" on the page (which will give us the same HTML code we would get from using `requests`), and see if we can find somewhere that this data is easily accessible.

Just searching for the title of the movie in the source (since it appears close to the data we want) we find the following:

![Screenshot of source code for Total Recall](https://pixelfed.social/storage/m/_v2/588554065884192073/bf0f52ff2-92677b/uMgKSygEYgiW/D5khczqYThR6JWahvNIpyTW8nl677Cq0SWC4kybC.png)

*How would we request the HTML code for the movie and select the `score-board` element?*

In [8]:
import requests
from bs4 import BeautifulSoup

resp = requests.get('https://www.rottentomatoes.com/m/total_recall')
resp.raise_for_status()

rt_soup = BeautifulSoup(resp.text, 'lxml')
scoreboard = rt_soup.find('score-board')
scoreboard

<score-board audiencescore="79" audiencestate="upright" data-qa="score-panel" data-scoredetailsmanager="scoreboard:scoreClick" id="scoreboard" mediatype="movie" rating="R" skeleton="panel" tomatometerscore="82" tomatometerstate="certified-fresh">
<h1 class="title" data-qa="score-panel-title" slot="title">Total Recall</h1>
<p class="info" data-qa="score-panel-subtitle" slot="info">1990, Sci-fi/Action, 1h 49m</p>
<a class="link" data-qa="tomatometer-review-count" href="/m/total_recall/reviews?intcmp=rt-scorecard_tomatometer-reviews" slot="critics-count">
            76 Reviews
        </a>
<a class="link" data-qa="audience-rating-count" href="/m/total_recall/reviews?type=user&amp;intcmp=rt-scorecard_audience-score-reviews" slot="audience-count">
            100,000+ Ratings
        </a>
<div id="tomatometer_sponsorship_ad" slot="sponsorship"></div>
</score-board>

In [9]:
audiencescore = int(scoreboard.get('audiencescore'))
rating = scoreboard.get('rating')
tomatometerscore = int(scoreboard.get('tomatometerscore'))
title = scoreboard.h1.text
year, genre, runtime = scoreboard.find('p', slot='info').text.split(', ')
title

'Total Recall'

---

Now let's go back to the problem of getting the initial list of "90s Science Fiction movies". For this task, we turn to [Wikidata](https://www.wikidata.org/wiki/Wikidata:Main_Page), a project of the Wikimedia Foundation (the folks behind Wikipedia). Wikidata stores information about the world in a form of "linked data". For example, it stores that the film [Back to the Future III](https://www.wikidata.org/wiki/Q230552) has a "country of origin" of [United States of America](https://www.wikidata.org/wiki/Q30). It also stores that the film has a genre of "science fiction film" and that, most importantly, it has a Rotten Tomatoes ID of "m/back_to_the_future_3".

Using the [Wikidata Query Service](https://query.wikidata.org/), we can construct a query that says "Give me the Rotten Tomatoes ID of all science fiction films made in the 90s". The construction of that query, which is in the SPARQL query language, will not be discussed further in this lesson. Perhaps just assume that if you were working on this project, your colleague prepared a list of Rotten Tomatoes ids for you.

https://w.wiki/76$A

<img width=600 alt="Screenshot of Wikidata Query Service with a query for 90s sci fi movies" src="https://pixelfed.social/storage/m/_v2/588554065884192073/bf0f52ff2-92677b/f6jKZtyCBm8Q/92LrBuaVJJuZPEKoSBIIvfGeP9mvRdLt6eiXl0R1.png"/>

Here is the code to programatically query the Wikidata Query Service and extract the IDs.

In [3]:
import requests

query = '''
SELECT DISTINCT ?item ?itemLabel ?rtid WHERE {
  ?item wdt:P136 wd:Q471839.
  ?item wdt:P577 ?pubdate.
  ?item wdt:P1258 ?rtid.
  FILTER((?pubdate >= "1990-01-01T00:00:00Z"^^xsd:dateTime) && ((?pubdate < "2000-01-01T00:00:00Z"^^xsd:dateTime)))
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
'''

resp = requests.post('https://query.wikidata.org/sparql',
              headers={'User-Agent': 'Noisebridge MovieBot 0.0.1/Audiodude <audiodude@gmail.com>'},
              data={
                'query': query,
                'format': 'json',
              }
)
resp.raise_for_status()

data = resp.json()
ids =[d['rtid']['value'] for d in data['results']['bindings']]

Once we have the IDs, we can loop over them and make the scraping request to each movie URL, storing the data in parallel lists that can be used to construct a Pandas DataFrame. Note that there is a bit of trial and error involved, in that sometimes the movie page doesn't have data for all of the things we are looking for, and we need to protect against that, but we might not know that until we try it. We could alternately wrap the `scrape_movie` call in a `try/except` block, so that if an error is thrown we just skip that movie.

In [ ]:
import urllib.parse
import time

from bs4 import BeautifulSoup
import pandas as pd
import requests
from tqdm import tqdm

def scrape_movie(url, data):
    resp = requests.get(url)
    if resp.status_code != 200:
        return

    rt_soup = BeautifulSoup(resp.text, 'lxml')
    scoreboard = rt_soup.find('score-board')
    audiencescore = scoreboard.get('audiencescore')
    # Protect against empty data
    if audiencescore:
        data['audiencescore'].append(int(audiencescore))
    else:
        data['audiencescore'].append(None)
        
    tomatometerscore = scoreboard.get('tomatometerscore')
    if tomatometerscore:
        data['tomatometerscore'].append(int(tomatometerscore))
    else:
        data['tomatometerscore'].append(None)
        
    data['rating'].append(scoreboard.get('rating'))
    data['title'].append(scoreboard.h1.text)
    
    info_parts = scoreboard.find('p', slot='info').text.split(', ')
    # If this text has three parts separated by comma, they are the fields below.
    # Otherwise, those fields are None.
    year = None
    genre = None
    runtime = None
    if len(info_parts) == 3:
        year, genre, runtime = info_parts
        
    data['year'].append(year)
    data['genre'].append(genre)
    data['runtime'].append(runtime)
     
data = {
    'audiencescore': [],
    'rating': [],
    'tomatometerscore': [],
    'title': [],
    'year': [],
    'genre': [],
    'runtime': [],
}

rt_base_url = 'https://www.rottentomatoes.com/'

# By wrapping the ids list in tqdm, we get a nice progress bar.
for id_ in tqdm(ids):
    time.sleep(1)
    url = urllib.parse.urljoin(rt_base_url, id_)
    scrape_movie(url, data)
    
df = pd.DataFrame(data)

In [18]:
df.sort_values('tomatometerscore', ascending=False).head()

,audiencescore,rating,tomatometerscore,title,year,genre,runtime
409,93.0,,100.0,Godzilla vs. Destoroyah,1995,Sci-fi,1h 43m
303,81.0,,100.0,Wings of Honneamise,1987,Sci-fi/Drama,1h 59m
254,90.0,PG,96.0,The Iron Giant,1999,Kids & family/Adventure,1h 26m
38,89.0,PG,95.0,The Truman Show,1998,Comedy/Drama,1h 42m
235,89.0,,95.0,Ghost in the Shell,1995,Action/Sci-fi,1h 25m


As with all taxonomies, it appears that the definition of "Science Fiction Film" on Wikidata is a bit....fluid.

---

## Additional considerations

### Javascript pages

Some webpages are constructed using Javascript almost exclusively. These pages might use something like the [React](https://react.dev/) framework to dynamically build the webpage after the page has first loaded. In this case, the returned HTML only contains a skeleton of the page and doesn't include the data we are after. There are potentially ways to interpret Javascript on a target page and extract the data anyways, but they are beyond the scope of this lesson.

### User-Agent

Some websites are sensitive to being scraped. The easiest thing those sites can do to disallow scraping it to check the **User-Agent** of the program that is being used to request the data. All browsers send a User-Agent **header** as part of their requests to websites. This is a string which identifies the browser version and operating system that is making the request. Currently, on Windows 10 with Chrome the value of my User-Agent header is:

```
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36
```

By default, requests will send a User-Agent of `Python-requests/x.y.z` where `x.y.z` is the version number of the library. A website might see this and decide it's not a real web browser and deny the request.

Since the User-Agent is data provided by the client (us), we can set it to anything we want. Setting the User-Agent to look like the request is coming from a web browser is called **User-Agent spoofing**.

In [ ]:
requests.get(
    'https://text.npr.org/',
    headers={
        'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
        '(KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'),
    }
)

If the website doesn't seem sensitive to User-Agent headers, another thing that you could do is send a User-Agent that identifies yourself or your bot. This is particularly courteous because it allows the operator of the website to identify you in the case that your bot is causing problems. Wikipedia, for example, has a [User-Agent policy](https://meta.wikimedia.org/wiki/User-Agent_policy) that requires bot operators and those running automated scripts to identify themselves in the User-Agent string.

```
MovieBot 1.0/<audiodude@gmail.com>
```

### Rate limiting

The most common mistake beginners make when performing web scraping is to "hammer" the target website. You might have a loop that requests every news article from the front page:

In [ ]:
def absolute_url(relative_url):
    return urllib.parse.urljoin(base_url, relative_url)

all_absolute_urls = [absolute_url(a.get('href')) for a in articles]
for url in all_absolute_urls:
    # This will make the requests as fast as possible, potentially multiple per second
    resp = requests.get(url)
    resp.text

As the comment says, if you run this code your computer will make network requests to the target website as fast as possible. Many websites will see this flood of requests and either recognize it as a scraping procedure, or even consider it a Denial of Service attack. This could lead to your IP address being blocked. The solution is to intentionally delay or **sleep** your code between requests. Although this might seem annoying it is both courteous to the website you're scraping and will prevent you from getting blocked.

In [ ]:
import time

def absolute_url(relative_url):
    return urllib.parse.urljoin(base_url, relative_url)

all_absolute_urls = [absolute_url(a.get('href')) for a in articles]
for url in all_absolute_urls:
    # Sleep for two seconds before making each request.
    time.sleep(2)
    resp = requests.get(url)
    resp.text

### IP Address

You've probably heard of IP Addresses. They're like your phone number on the Internet. Although most home internet providers use rotating IP addresses, it is likely that your home IP address will stay the same for a relatively long period (measured in weeks).

If you get banned from scraping a website, they will most likely do so by using your IP address, so that when they see a request from your IP, they will drop it or return an error code. This can look like requests that hang indefinitely.

There are web scraping services that will allow you to connect to a **proxy**, which will cause your IP address to be different every time you make a request to the website you are scraping. This will let you avoid IP bans.

### Pagination

Sometimes, the data you want will be spread across multiple "pages" of an app. We have a couple of options in this case. We could inspect the HTML of the page for the link to "next page" and use the URL of that link to visit the next page of results. We could also programatically construct the URLS by figuring out the pagination scheme. For example if the urls are like:

```
https://example.com/forum/posts?page=1
```

We could increment the page value to 2, 3, etc. Then we could request each of these URLs in order to scrape the data.

### Bypassing HTML

It's possible that a page you are trying to scrape doesn't use Javascript exclusively (see above) but might use it incrementally, for example with some "infinite scroll" behavior on a search results page. In these cases, you can often use the "Network" tab of the excellent [Chrome DevTools](https://developer.chrome.com/docs/devtools/) to figure out what network request the page is making. For example, if you are on `https://example.com/search?q=cats`, you might see that the page is making requests to `https://api.example.com/v1/search?q=cats&page=2`. In this case, you can run your scraper against the "backend" URL directly to retrieve the data.

---

And that's it! Scraping is a large and varied topic, but we've touched on many of the major processes and concerns. Real companies use web scraping every day to collect data that they use in their products, or to survey competitors. You can imagine that the basic techniques in this article could be expanded upon further to build production ready scraping pipelines.